#5. Calibração de modelos

Realize a transformação do alvo descrita na parte 3 para enquadrar o problema
como classificação binária.

Investigue o grau de calibração do modelo resultante e, conforme for o caso, aplique alguma técnica para calibrar os
resultados do modelo.

Conjuntos de dados: A652.pickle

Número de variáveis independentes: 7

Variável dependente: "precipitation"

Modelo utilizado: "GradientBoosting" e "RandomForest"

##5.1 Importação de bibliotecas

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.calibration import CalibratedClassifierCV


##5.2 Importação dos dados

Importação dos arquivos de treino e teste disponibilizados no github através da biblioteca pandas

In [3]:
A652_train_E5 = pd.read_parquet("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/parquet/A652_train.parquet.gzip")
A652_test_E5 = pd.read_parquet("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/parquet/A652_test.parquet.gzip")
A652_val_E5 = pd.read_parquet("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/parquet/A652_val.parquet.gzip")

##5.3 Pre-processamento de dados

Método two-way holdout: Agrupando dados de treinamento e validação

In [4]:
A652_train_E5 = pd.concat([A652_train_E5, A652_val_E5])

Separando a variável dependente (y) das features (X) nos dados de treino e teste e preparando os dados para o modelo de classificação, transformando os valores numéricos da variavél "y" em valores categóricos.

In [5]:
def data_preparation_E5(data):

  data['categorical_precipitation'] = data['precipitation'].apply(lambda x: 0 if x== 0 else 1)

  X = data.drop(['categorical_precipitation', 'precipitation'], axis = 1)
  y = data['categorical_precipitation']

  return(X,y)

In [6]:
X_A652_train_E5, y_A652_train_E5 = data_preparation_E5(A652_train_E5)
X_A652_test_E5, y_A652_test_E5 = data_preparation_E5(A652_test_E5)

##5.4 Criando/treinando/avaliando modelo

In [7]:
data_original_E5 = {
    'A652' : (X_A652_train_E5 , y_A652_train_E5, X_A652_test_E5 , y_A652_test_E5)
}

Treinando o modelo "GradientBoostingClassifier" utilizando a método de calibração das probabilidades de predição: "Logloss"

In [8]:
model_gbc_E5 = GradientBoostingClassifier()
model_gbc_E5.fit(X_A652_train_E5, y_A652_train_E5)
y_gbc_test_prob_E5 = model_gbc_E5.predict_proba(X_A652_test_E5)[:,1]
prob_true_gbc, prob_pred_gbc = calibration_curve(y_A652_test_E5, y_gbc_test_prob_E5, n_bins = 10 )


Avaliando a curva de calibração graficamente.

In [9]:
chart_df_gbc = pd.DataFrame({
    "actuals": prob_true_gbc,
    "predicted": prob_pred_gbc,
    "expected": prob_pred_gbc
})
fig = px.line(
        data_frame = chart_df_gbc,
        markers = True,
        x = "predicted",
        y = ["actuals", "expected"],
        template = "plotly_dark")
fig.update_layout(
        title  = {"text": "Calibration Plot : GradientBoostingClassifier(Logloss_Method)", "y": 0.95, "x": 0.5},
        xaxis_title="Predicted Probability",
        yaxis_title="Actual Probability",
        font = dict(size=15)
)
fig.show()

Treinando o modelo "RandomForestClassifier" utilizando a método de calibração das probabilidades de predição: "Sigmoid"

In [10]:
model_rfc = CalibratedClassifierCV(RandomForestClassifier(random_state=36))
model_rfc_calibrated_sigmoid =CalibratedClassifierCV(model_rfc , method='sigmoid', cv = 3)
model_rfc_calibrated_sigmoid.fit(X_A652_train_E5, y_A652_train_E5)

y_rfc_test_prob_E5 = model_rfc_calibrated_sigmoid.predict_proba(X_A652_test_E5)[:,1]
prob_true_rfc, prob_pred_rfc = calibration_curve(y_A652_test_E5, y_rfc_test_prob_E5, n_bins = 10 )

Avaliando a curva de calibração graficamente.

In [11]:
chart_df = pd.DataFrame({
    "actuals": prob_true_rfc,
    "predicted": prob_pred_rfc,
    "expected": prob_pred_rfc
})
fig = px.line(
        data_frame = chart_df,
        markers = True,
        x = "predicted",
        y = ["actuals", "expected"],
        template = "plotly_dark")
fig.update_layout(
        title  = {"text": "Calibration Plot : RandomForestClassifier(Sigmoid_Method)", "y": 0.95, "x": 0.5},
        xaxis_title="Predicted Probability",
        yaxis_title="Actual Probability",
        font = dict(size=15)
)
fig.show()